## MSDS 452
## Assignment 1

* ### Here I am selecting a Wikipedia page through a random generator of wikipedia pages.
* ### However, one may start with a predifined Wikipedia page of one's own choice. In this case, one needs to modify Cell 7 below.
* ### Subsequently, the goal is to find the set of hyperlinks of the original (random) page to other Wikipedia webpages and the graph (directed network) which is formed by all the hyperlinks among the latter webpages. 
* ### To be able to controll the size of this graph, we are setting the minimum and the maximum number of hyperlinks from the original (random) page into a fixed interval of values (parameters n and N below).
* ### At the very end, after running all cells of this notebooks, students are asked to write a short summary of the contents of each one of the webpages to which the original (random) Wikipedia page points.

In [ ]:
# There are two wikipedia module in Python:

# To install the first one:
# !pip install wikipedia-api

# To install the second:
# !pip install wikipedia

# Here, we will be using both modules

In [ ]:
# To install pygraphviz with conda, 
# you need to run the following IN A TERMINAL (after commenting it out):

# conda install -c alubbock pygraphviz

In [ ]:
# To install pygraphviz with pip, 
# you need to run the following (after commenting it out):

# !pip install graphviz
# !pip install python-graphvis
# !pip install pygraphviz

In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
import wikipedia
import random
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
import warnings; warnings.simplefilter('ignore')

In [ ]:
n=30  # minimum number of hyperlinks of the selected wikipedia page
N=51  # maximum number of hyperlinks of the selected wikipedia page

### Random Selection of a Wikipedia Page

In [ ]:
# If this cell runs very slowly, take lower values of n and N.

while True:
    try:
        page=wiki_wiki.page(wikipedia.random())
#         page=wikipedia.page(wikipedia.random())
    except Error as e:
#     except wikipedia.DisambiguationError as e:
        print(e)
        pass
    except wiki_wiki.exceptions.PageError as e:
#     except wikipedia.exceptions.PageError as e:
        print(e)
        pass
    hl=sorted([w.replace(" ","_") for w in page.links])
    if len(hl)>n and len(hl)<N:
        break
        
p=sorted([w.replace(" ","_") for w in page.links])

print("The %i hyperlinks from the (randomly) selected '%s' Wikipedia page are: \n" %(len(p),page.title))
for i in range(len(p)):
    print("%i. %s" %(i+1,p[i]))

### Using a Predefined Wikipedia Page

In [ ]:
# # IF ONE WANTS TO USE A PREDEFINED WEBPAGE, 
# # one should uncomment and run the following lines
# # Here, I've considered as predefined Wikipedia page the page of the
# # 'United_States_women%27s_national_soccer_team' but you may take
# # instead anything you like. Notice, that subsequently I am filtering
# # a random sample of 20 hyperlinks. This might need to change if the
# # page you have selected has fewer hyperlinks.

# page=wikipedia.page('United_States_women%27s_national_soccer_team') #'https://en.wikipedia.org/wiki/United_States_women%27s_national_soccer_team'

# p=random.sample(sorted([w.replace(" ","_") for w in page.links]),20)

# print("The %i hyperlinks from the (randomly) selected '%s' Wikipedia page are: \n" %(len(p),page.title))
# for i in range(len(p)):
#     print("%i. %s" %(i+1,p[i]))

In [ ]:
eds=[]
for pp in p:
    try:
        ppp=sorted([w.replace(" ","_") for w in wiki_wiki.page(pp).links])
#         ppp=sorted([w.replace(" ","_") for w in wikipedia.page(pp).links])
    except Error as e:
#     except wikipedia.exceptions.PageError as e:
        print(e)
        pass
    ppp=[x for x in ppp if x!=pp]
    ih=set(ppp).intersection(set(p))
    if len(ih)>0:
        print("%s has %i hyperlinks to webpages: \n %s \n" %(pp,len(ih),", ".join(list(ih))))
        for q in ih:
            eds.append((pp,q))

In [ ]:
# In case you could not install pygraphviz, replace the command bellow:
# pos=graphviz_layout(G)
# by 
# pos=nx.spring_layout(G,k)
# where you set k equal to 1. or any float less or larger that 1.
# Try a few assignments of k until you find the plot that looks more satisfactory.

In [ ]:
G=nx.DiGraph()
G.add_edges_from(eds)
# print(len(G.nodes()),len(G.edges()))

pos=graphviz_layout(G)

# pos=nx.spring_layout(G,1.)

plt.figure(figsize=(10,10));
nodes = nx.draw_networkx_nodes(G, pos, node_color="g", alpha=0.2)
nx.draw_networkx_edges(G, pos,arrowsize=20, edge_color="b", alpha=0.7)
nx.draw_networkx_labels(G, pos)
plt.axis('off');
ss="The graph of the %i Wikipedia webpages which are \n hyperlinks of the Wikipedia page '%s'" %(len(G.nodes()),page.title)
plt.title(ss,fontsize=15);
plt.margins(x=0.3, y=0) 

<p style="font-size:20pt"><span style="color:green">Repeat the previous cells so that you get one</span> <span style="color:blue">connected</span> <span style="color:green">and one</span> <span style="color:red">disconnected</span> <span style="color:green">Wikipedia graph!</span></p>

In [ ]:
import random, math, numpy as np, matplotlib.pyplot as plt, pycountry, awoc, pandas as pd  
from collections import Counter
import networkx as nx
from networkx.algorithms import bipartite
from networkx.algorithms import community
import community as louvain

import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph

import bokeh
from bokeh.io import push_notebook, show, output_notebook, save
import bokeh.plotting as bp
from bokeh.plotting import figure, save, output_file, show #, from_networkx
from bokeh.models import (ColumnDataSource, LabelSet, Label, BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,)
from holoviews.element.graphs import layout_nodes
# bokeh.sampledata.download()
from bokeh.sampledata.airport_routes import routes, airports

output_notebook()
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh', 'matplotlib')
from holoviews.operation import  gridmatrix
from holoviews.operation.datashader import datashade, bundle_graph
from holoviews import Graph, Nodes
from holoviews.plotting.bokeh import GraphPlot, LabelsPlot
import hvplot.networkx as hvnx
import hvplot.pandas

def hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,title,bundled,nodelabels,xoffset,yoffset,
              arrowhead_length=0.04,selection_mode='nodes',selection_policy="nodes",
              edge_hover_line_color='green',node_hover_fill_color='red',
              fontsize={'title': '9pt'},text_font_size='8pt', text_color='black',bgcolor='white'):
    if nx.is_directed(G)==True:
        if ctype==1:
            communities_generator = community.girvan_newman(G)
            top_level_communities = next(communities_generator)
            next_level_communities = next(communities_generator)
            lc=sorted(sorted(map(sorted, next_level_communities)), key=len,reverse=True)
            partition={n:i for i,c in enumerate(lc) for n in c }
            print("No. of Girvan-Newman communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Girvan_Newman_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['in-degree'] = 0
                    G.nodes[n]['in-neighbors'] = ""
                    G.nodes[n]['out-degree'] = 0
                    G.nodes[n]['out-neighbors'] = ""
                else:    
                    G.nodes[n]['in-degree'] = G.in_degree(n)
                    G.nodes[n]['in-neighbors'] = ", ".join([str(x) for x in list(G.predecessors(n))])
                    G.nodes[n]['out-degree'] = G.out_degree(n)
                    G.nodes[n]['out-neighbors'] = ", ".join([str(x) for x in list(G.successors(n))])
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color='Girvan_Newman_community',node_cmap=node_cmap)
            graph.opts(padding=0.15)
            if bundled==0:                
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')   
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
    else:
        if ctype==1:
            communities_generator = community.girvan_newman(G)
            top_level_communities = next(communities_generator)
            next_level_communities = next(communities_generator)
            lc=sorted(sorted(map(sorted, next_level_communities)), key=len,reverse=True)
            partition={n:i for i,c in enumerate(lc) for n in c }
            print("No. of Girvan-Newman communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Girvan_Newman_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['degree'] = 0
                    G.nodes[n]['neighbors'] = ""
                else:    
                    G.nodes[n]['degree'] = G.degree(n)
                    G.nodes[n]['neighbors'] = ", ".join([str(x) for x in list(G.neighbors(n))])
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color='Girvan_Newman_community',node_cmap=node_cmap)
            graph.opts(padding=0.15)
            if bundled==0:
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')   
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
        if ctype==0:
            partition=louvain.best_partition(G)
            print("No. of Louvain communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Louvain_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['degree'] = 0
                    G.nodes[n]['neighbors'] = ""
                else:    
                    G.nodes[n]['degree'] = G.degree(n)
                    G.nodes[n]['neighbors'] = ", ".join([str(x) for x in list(G.neighbors(n))])
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color="Louvain_community",node_cmap=node_cmap)
            graph.opts(padding=0.15)
            if bundled==0:                
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')   
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length,tools=['hover','box_select'])
#                 graph.select(selection_mode,circle='circle15')
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph

### 1. Holoviews visualization: non-bundled edges and nodes with labels

In [ ]:
ctype=1
bundled=0
nodelabels=1
width=1000
height=1000
pos=graphviz_layout(G)
node_size=0.5*(15+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title="Wikipedia Graph of Hyperlinks"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'},
text_font_size='8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-10

hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,title,bundled,nodelabels,xoffset,yoffset,
              arrowhead_length=0.04,selection_mode='nodes',selection_policy="nodes",
              edge_hover_line_color='green',node_hover_fill_color='red',
              fontsize={'title': '9pt'},text_font_size='8pt', text_color='black',bgcolor='white')

### 2. Holoviews visualization: bundled edges and nodes without labels

In [ ]:
ctype=1
bundled=1
nodelabels=0
width=1000
height=1000
pos=graphviz_layout(G)
node_size=0.5*(15+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title="Wikipedia Graph of Hyperlinks"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'},
text_font_size='8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-10

hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,title,bundled,nodelabels,xoffset,yoffset,
              arrowhead_length=0.04,selection_mode='nodes',selection_policy="nodes",
              edge_hover_line_color='green',node_hover_fill_color='red',
              fontsize={'title': '9pt'},text_font_size='8pt', text_color='black',bgcolor='white')